# Projet Gustave Malachane Kilian Violet

In [8]:
!py -m pip install requests

In [9]:
import urllib.request
import json
import pandas as pd
import requests
import shutil
import os

def download_image(url, name):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open("./images/"+name+".jpg", "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code


url = "https://query.wikidata.org/sparql?query=Select%20%3Fvolcan%20%3Fimage%20%3Flabel_pays%20%3Fhauteur%20%3Flabel%20%7B%0A%20%20%3Fvolcan%20wdt%3AP31%20wd%3AQ169358%3B%0A%20%20%20%20%20%20%20%20wdt%3AP18%20%3Fimage%3B%0A%20%20%20%20%20%20%20%20wdt%3AP17%20%3Fpays%3B%0A%20%20%20%20%20%20%20%20wdt%3AP2660%20%3Fhauteur%3B%0A%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3Flabel.%0A%20%20%0A%20%20%3Fpays%20rdfs%3Alabel%20%3Flabel_pays.%0A%20%20%0A%20%20FILTER(lang(%3Flabel)%3D%22en%22%20%26%26%20lang(%3Flabel_pays)%3D%22en%22)%0A%7D%0A%0ALIMIT%20150&format=json"
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))

array = []
metadata_list = []
i = 1
for data in responsedata["results"]["bindings"]:

    metadata = {
        "nom": data["label"]["value"],
        "hauteur": data["hauteur"]["value"],
        "pays": data["label_pays"]["value"]            
    }

    metadata_list.append(metadata)
    response = download_image(str(data["image"]["value"]).replace("%20", "_").replace("%C3%A1", "a"), metadata["nom"])
    
    i += 1
    
dataframe = pd.DataFrame(array, columns=["label", "image"])

with open("metadata.json", "w", encoding="utf-8") as json_file:
    json.dump(metadata_list, json_file, indent=4, ensure_ascii=False)

## Récupération des données exifs des images

Ici grâce à la librairie PIL & exifread, nous pouvons accéder aux données exifs de l'image. elles sont ensuite stockées dans le fichier JSON metadata.json

In [10]:
!py -m pip install exifread

In [11]:
import os
import json
from PIL import Image
import exifread
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image


def get_dominant_colors(image_path, k=5):
    """ Extrait les k couleurs dominantes d'une image avec K-Means """
    with Image.open(image_path) as img:
        img = img.resize((100, 100))  # Réduire la taille pour accélérer le clustering
        img_array = np.array(img).reshape(-1, 3)  # Convertir en tableau de pixels

    # Clustering des couleurs
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(img_array)

    # Convertir les couleurs en hexadécimal
    colors = ['#%02x%02x%02x' % tuple(map(int, color)) for color in kmeans.cluster_centers_]
    return colors

# Dossier contenant les images
image_folder = "images"

# Parcourir chaque fichier image
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    try:
        # Ouvrir l'image avec PIL
        with Image.open(image_path) as img:
            width, height = img.size
            format_img = img.format

            colors = get_dominant_colors(image_path)

            # Déterminer l'orientation
            if width > height:
                orientation = "Paysage"
            elif height > width:
                orientation = "Portrait"
            else:
                orientation = "Carré"

        # Lire les métadonnées EXIF
        with open(image_path, "rb") as img_file:
            tags = exifread.process_file(img_file, details=False)
            date_creation = tags.get("EXIF DateTimeOriginal", "Inconnu")
            camera_model = tags.get("Image Model", "Inconnu")

        # Stocker les métadonnées dans un dictionnaire
        metadataIndex = next((i for i, md in enumerate(metadata_list) if md["nom"]+".jpg" == image_name), -1)
        
        exif = {
            "nom": image_name,
            "chemin": image_path,
            "taille": f"{width}x{height}",
            "format": format_img,
            "orientation": orientation,
            "date_creation": str(date_creation),
            "modele_camera": str(camera_model),
            "colors": colors,
        }

        metadata_list[metadataIndex]["image"] = exif

    except Exception as e:
        print(f"⚠️ Erreur avec {image_name} : {e}")

# Sauvegarder les métadonnées en JSON
with open("metadata.json", "w", encoding="utf-8") as json_file:
    json.dump(metadata_list, json_file, indent=4, ensure_ascii=False)

print("\n Métadonnées enregistrées dans metadata.json")


⚠️ Erreur avec Mount Ragang.jpg : Image size (768777555 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.

 Métadonnées enregistrées dans metadata.json


### QUESTION : Comment prévoyez-vous de traiter ces tags ? Est-il possible d'automatiser ce processus ?

On peut utiliser K-means pour analyser et regrouper les couleurs dominantes.
Ici les données présentes dans le fichier metadata.json proviennent essentiellement des exifs récupérée grâce à la librairie exifread.
Il y aussi une partie des données comme la hauteur, le pays, le nom de l'image et l'image en elle-même qui proviennent de Wikidata. 
Pour les autres tags, on peut demander aux utilisateurs de remplir les tags par images.

In [17]:
import json
import random

# Charger les volcans depuis un fichier JSON
with open("metadata.json", "r", encoding="utf-8") as f:
    volcans = json.load(f)

# Vérifier le nombre total de volcans
nombre_total_volcans = len(volcans)
print(f"Nombre total de volcans chargés : {nombre_total_volcans}")

# Génération de 50 utilisateurs fictifs
utilisateurs = [
    {"id": i, "nom": f"Utilisateur_{i}", "volcans_consultés": []}
    for i in range(1, 51)
]

# Attribution aléatoire des volcans consultés (en enregistrant uniquement le nom)
for utilisateur in utilisateurs:
    nb_volcans = random.randint(5, 20)  # Chaque utilisateur consulte entre 5 et 20 volcans
    volcans_choisis = random.sample(volcans, min(nb_volcans, nombre_total_volcans))
    
    # Stocker uniquement le nom des volcans
    utilisateur["volcans_consultés"] = [volcan["nom"] for volcan in volcans_choisis]

# Sauvegarde des données dans un fichier JSON
with open("utilisateurs_volcans.json", "w", encoding="utf-8") as f:
    json.dump(utilisateurs, f, indent=4, ensure_ascii=False)

# Affichage d'un aperçu des 5 premiers utilisateurs
print(json.dumps(utilisateurs[:5], indent=4, ensure_ascii=False))  # Afficher les 5 premiers utilisateurs


Nombre total de volcans chargés : 130
[
    {
        "id": 1,
        "nom": "Utilisateur_1",
        "volcans_consultés": [
            "Mount Nikkō-Shirane",
            "Mount Pavlof",
            "Mentolat",
            "Mount St. Helens",
            "Imbabura Volcano",
            "Mount Ragang"
        ]
    },
    {
        "id": 2,
        "nom": "Utilisateur_2",
        "volcans_consultés": [
            "Lawu Mountain",
            "Gunung Merbabu",
            "Arjuno-Welirang",
            "Licancabur",
            "Mount Ragang",
            "Mount St. Helens",
            "Mount Hood",
            "La Soufrière",
            "Mount Pelée",
            "Mayon Volcano",
            "The Quill",
            "Iztaccihuatl",
            "Mount Vesuvius",
            "Antisana volcano",
            "Mount Kalatungan"
        ]
    },
    {
        "id": 3,
        "nom": "Utilisateur_3",
        "volcans_consultés": [
            "Gunung Karang",
            "Rinjani",
      